In [1]:
import json

with open('connectivity_result.json', 'r', encoding='utf-8') as f:
    datas=json.load(f)

In [3]:
with open('connectivity_test.json','r') as f:
    gt = json.load(f)

In [4]:
print(len(datas))
print(len(gt))
print(gt[0])

1861
1861
{'input_prompt': 'Determine if there is a path between two nodes in the graph. Note that (i,j) means that node i and node j are connected with an undirected edge.\nGraph: (0,12) (0,13) (0,2) (0,14) (0,23) (0,8) (0,1) (0,25) (0,27) (0,10) (1,29) (1,13) (1,19) (1,28) (1,23) (1,30) (1,25) (1,24) (2,12) (2,5) (2,14) (2,17) (2,23) (3,7) (3,12) (3,19) (3,23) (3,8) (3,30) (3,25) (3,24) (4,33) (5,20) (5,23) (5,30) (6,15) (6,11) (7,32) (7,17) (7,23) (7,8) (7,10) (8,29) (8,12) (8,32) (8,20) (8,19) (8,17) (8,30) (9,29) (9,28) (9,30) (9,24) (10,13) (10,20) (10,14) (10,22) (11,31) (12,13) (12,23) (12,27) (12,26) (13,29) (13,32) (13,14) (13,28) (13,30) (13,24) (14,17) (14,30) (14,22) (14,27) (15,16) (15,31) (15,33) (17,32) (17,20) (17,19) (17,27) (17,24) (17,26) (18,21) (20,28) (20,30) (20,25) (21,31) (22,32) (22,28) (22,30) (23,28) (23,25) (23,27) (24,29) (24,28) (24,26) (25,32) (26,29) (26,27) (27,32) (27,30) (28,32) (28,30)\nQ: Is there a path between node 14 and node 2?\nA:', 'answer':

In [5]:
import re
import io
import sys
print(len(gt))
print(len(datas))

1861
1861


In [6]:
print(gt[2]['input_prompt'])
print(datas[2]['code'])
print(gt[2]['answer'])


Determine if there is a path between two nodes in the graph. Note that (i,j) means that node i and node j are connected with an undirected edge.
Graph: (0,1) (0,7) (0,6) (1,7) (1,5) (1,6) (2,8) (2,4) (2,3) (3,8) (5,7) (6,7)
Q: Is there a path between node 1 and node 7?
A:
```python
import re
import networkx as nx

# Graph description
graph = "(0,1) (0,7) (0,6) (1,7) (1,5) (1,6) (2,8) (2,4) (2,3) (3,8) (5,7) (6,7)"

# Regular expression to extract edges
pattern = r"\((\d+),(\d+)\)"

# Create an undirected graph
G = nx.Graph()

# Find all matches in the graph
matches = re.findall(pattern, graph)

# Add edges to the graph
for i, j in matches:
    G.add_edge(int(i), int(j))

# Check if there is a path between node 1 and node 7
source = 1
target = 7
path_exists = nx.has_path(G, source, target)
print(path_exists)
```
yes.


In [14]:
error_cnt = 0
correct_cnt = 0
for i, data in enumerate(datas):
    text = data['code']
    code = re.sub(r'^```python\n|```$', '', text, flags=re.MULTILINE)
    ans = gt[i]['answer']

    # 创建一个 StringIO 对象来捕获输出
    output = io.StringIO()

    # 重定向 sys.stdout
    old_stdout = sys.stdout
    sys.stdout = output

    try:
        exec(code, globals(), globals())
    except:
        sys.stdout = old_stdout
        #print(code)
        error_cnt = error_cnt + 1
        continue
    finally:
        sys.stdout = old_stdout
    

    # 获取输出内容
    output_content = output.getvalue()
    output_content = str(output_content).strip()
    # print("Captured Output:")
    # print(output_content)
    if output_content == 'True':
        output_content = 'yes'
    elif output_content == 'False':                                 
        output_content = 'no'
    
    # print(f'llm={output_content} gt={ans}')
        
    if output_content.strip() == str(ans).split('.')[0].strip():
        correct_cnt = correct_cnt + 1
        # print(gt[int(i)]['input_prompt'])
        # print(code)
    else:
        print(gt[int(i)]['input_prompt'])
        print(output_content,ans)
        print(code)

In [15]:
print(error_cnt)
print(correct_cnt)

224
1637


In [16]:
print(correct_cnt/len(gt))

0.8796346050510478


In [25]:
import re
import networkx as nx

# Graph description
graph = "The nodes are numbered from 0 to 12, and the edges are: (0, 8) (0, 12) (1, 4) (1, 12) (2, 11) (2, 10) (3, 6) (3, 7) (3, 12) (3, 9) (5, 11) (7, 10) (7, 9) (8, 12)."

# Regular expression to extract directed edges
pattern = r"\((\d+), (\d+)\)"

# Create a directed graph
G = nx.Graph()

# Find all matches in the text
matches = re.findall(pattern, graph)

# Add edges to the graph
for i, j in matches:
    G.add_edge(int(i), int(j))

# Calculate shortest path length
length = nx.shortest_path_length(G, source=12, target=5)
print(length)


6
